In [21]:
from scipy.stats import spearmanr, rankdata
from scipy.spatial.distance import pdist, squareform

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import shutil
import time
import _pickle as cp
import sys
from astropy.stats import median_absolute_deviation
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler

from PIL import Image

## đọc dữ liệu

In [22]:
# Đọc dữ liệu từ file CSV
full_data = pd.read_csv('./ddos/train csv/LDAP-training.csv')

In [23]:
full_data.columns

In [24]:
# chọn đặc trưng

labels = full_data['Label']

selected_columns = ['Total Fwd Packets', 'Total Backward Packets',
       'Fwd Packets Length Total', 'Bwd Packets Length Total',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Avg Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
       'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate',
       'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'Init Fwd Win Bytes', 'Init Bwd Win Bytes',
       'Fwd Act Data Packets', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

data = full_data[selected_columns]

len(selected_columns)

75

## tách dataframe theo các nhãn

In [25]:

# tách tập dữ liệu theo label

## Nhóm dữ liệu theo cột 'Label'
grouped = full_data.groupby('Label')

## Tạo dictionary để lưu các DataFrame tương ứng với từng nhãn
dfs = {label: group for label, group in grouped}

## Kiểm tra một số DataFrame vừa tạo ra
# for label, df_label in dfs.items():
    # print(f"DataFrame cho label {label}:")
    # print(df_label)
    # print("\n")

## chuyển sang hình ảnh

- B1: chuẩn hóa dữ liệu về [0;1]
- lưu ảnh và thư mục của từng nhãn

#### hàm chuyển đổi sang hình ảnh

In [26]:
def convert_to_img(data_normalized, label):
    # Chuyển đổi mỗi hàng thành hình ảnh

    folder_path = f'./Results/Test_3/{label}'

    # Kiểm tra nếu thư mục đã tồn tại
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    for index, row in data_normalized.iterrows():

        image_array = np.array(row).reshape((5, 15))  # hiện tại chọn 75 đặc trưng nên chọn reshape với kích thước 5x15 = 75
        image_array = np.nan_to_num(image_array)  # Thay thế các giá trị không hợp lệ bằng 0
        image = Image.fromarray((image_array * 255).astype(np.uint8)) # nhân 255 để chuyển sang giá trị RGB
       
        image = image.resize((224, 224))  # Chuyển đổi kích thước hình ảnh thành 224x224 (thường train model dùng kích thước này)
        image.save(f'{folder_path}/image_{index}.png')

# Đọc và hiển thị một hình ảnh mẫu
# image = Image.open('./Results/Test_3/LDAP-training/image_0.png')
# image.show()

In [28]:

for label, df_label in dfs.items():

    # chọn đặc trưng
    data_features = df_label[selected_columns]


    ''' tiền xử lý '''
    # Chuyển đổi các giá trị thành số, các giá trị không thể chuyển đổi sẽ trở thành NaN
    # data = data.apply(pd.to_numeric, errors='coerce')

    # Loại bỏ các hàng chứa giá trị NaN
    # data = data.dropna()

    ''' chuẩn hóa dữ liệu Min-Max Scaling '''
    # data_min_max_scaled = (data_features - data_features.min()) / (data_features.max() - data_features.min())
    # data_normalized = data_min_max_scaled
    # convert_to_img(data_normalized, label)


    ''' chuẩn hóa dữ liệu Standardization '''
    data_features = np.log1p(data_features + 1)
    data_standardized = (data_features - data_features.mean()) / data_features.std()
    data_normalized = data_standardized
    convert_to_img(data_normalized, label)  


    ''' chuẩn hóa dữ liệu Log Transformation '''
    # ddata_log_transformed = np.log1p(data_features)
    # data_normalized = ddata_log_transformed
    # convert_to_img(data_normalized, label)  

    ''' kết hợp tất cả phương pháp chuẩn hóa '''
    # data_min_max_scaled = (data_features - data_features.min()) / (data_features.max() - data_features.min())

    # data_features = np.log1p(data_min_max_scaled + 1)
    # data_standardized = (data_features - data_features.mean()) / data_features.std()

    # ddata_log_transformed = np.log1p(data_standardized)
    # data_normalized = ddata_log_transformed
    # convert_to_img(data_normalized, label)  
    

c:\Users\NewTun\Desktop\Data to image test\test\.venv\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: divide by zero encountered in log1p
  result = func(self.values, **kwargs)
c:\Users\NewTun\Desktop\Data to image test\test\.venv\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)
c:\Users\NewTun\Desktop\Data to image test\test\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\NewTun\Desktop\Data to image test\test\.venv\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: divide by zero encountered in log1p
  result = func(self.values, **kwargs)
c:\Users\NewTun\Desktop\Data to image test\test\.venv\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)
c:\